## Instllations



To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Llama-3 8b is trained on a crazy 15 trillion tokens! Llama-2 was 2 trillion.**

Use our [Llama-3 8b Instruct](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing) notebook for conversational style finetunes.

In [29]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

## Utils

In [30]:
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score
from collections import Counter
import numpy as np
import re
import os

def extract_float(text):
    match = re.search(r"[-+]?\d*\.\d+", text)
    if match:
        return float(match.group(0))
    else:
        return None

def r44(value): return f"{round(value, 3):.3f}"

def r55(value): return f"{round(value, 4):.4f}"

def rrmse(targets, predictions):
    return np.sqrt(((predictions - targets) ** 2).mean())/np.mean(targets)

def metrics(y_test, y_pred, roundd = False):
    rrmse_ = rrmse(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** (1/2)
    r2 = r2_score(y_test, y_pred)
    return rrmse_, rmse, r2

def stid(source_name, target_name):
    return (source_name[0:3] + '-' + target_name[0:3]).lower()

def np_ratio(arr):
    C = Counter(arr)
    return C[0]/C[1]

def evaluator(test_data, model):
    generated_output = []
    real_output = []
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    for testcases in tqdm(test_data):
        inputs = tokenizer(
        [
          alpaca_prompt.format(
              testcases["instruction"], # instruction
              testcases["input"], # input
              "", # output - leave this blank for generation!
          )
      ], return_tensors = "pt").to("cuda")

        outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

        outputs = tokenizer.batch_decode(outputs)[0]

        outputs = extract_float(outputs.split("### Response:")[1].strip())
        benchmark = extract_float(testcases["output"])

        generated_output.append(outputs)
        real_output.append(benchmark)

    valid_indices = [i for i, output in enumerate(generated_output) if output is not None]
    generated_output_filtered = np.array([generated_output[i] for i in valid_indices])
    real_output_filtered = np.array([real_output[i] for i in valid_indices])
    print('validated generations:', len(generated_output_filtered)/len(test_data))

    rrmse, rmse, r2 = metrics(real_output_filtered, generated_output_filtered)
    print(rrmse, rmse, r2)
    return real_output_filtered, generated_output_filtered

## Getting Models and Trainer

In [31]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [32]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Data/Model Prep

<a name="Data"></a>
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [33]:
import pickle
from datasets import Dataset

with open('./serialized_chi_san_train.pkl', 'rb') as f:
  serialized_chi_train = pickle.load(f)

with open('./serialized_chi_test.pkl', 'rb') as f:
  serialized_chi_test = pickle.load(f)

print(len(serialized_chi_train), len(serialized_chi_test))

serialized_chi_train_dataset = Dataset.from_dict({key: [d[key] for d in serialized_chi_train] for key in serialized_chi_train[0]})
serialized_chi_test_dataset = Dataset.from_dict({key: [d[key] for d in serialized_chi_test] for key in serialized_chi_test[0]})

with open('./serialized_san_train.pkl', 'rb') as f:
  serialized_san_train = pickle.load(f)

with open('./serialized_san_test.pkl', 'rb') as f:
  serialized_san_test = pickle.load(f)

print(len(serialized_san_train), len(serialized_san_test))

serialized_san_train_dataset = Dataset.from_dict({key: [d[key] for d in serialized_san_train] for key in serialized_san_train[0]})
serialized_san_test_dataset = Dataset.from_dict({key: [d[key] for d in serialized_san_test] for key in serialized_san_test[0]})

6804 3710
7172 1558


In [34]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context.\
                   Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{} Is the water [safe] or [unsafe], select one based on the information.

### Response:
{}"""



EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = serialized_chi_train_dataset.map(formatting_prompts_func, batched = True,)


Map:   0%|          | 0/6804 [00:00<?, ? examples/s]

In [35]:
dataset[0]

{'instruction': 'We want to predict the log10 of bacteria concentration level in the beaches. you will receive environmental information and you should answer with a precise float number between -0.200 and 5.000',
 'input': 'Day of year: 153.0000. Wave Height (meters): 0.0769. Water Temperature (degree Celcius): 21.1667. Tide: 176.9059. Solar radiance (watts/meter squared): 0.0000. Did we have rain the past 3 days? Yes. Did we have rain the past 7 days? No. Log10 of cummulative rain in the last 3 days in inches: 0.0000. Log10 of cummulative rain in the last 3 days in inches: -0.5476. Is tide value more than the mean tide? No. Alongshire wind speed (meters/second): -0.0000. offshore wind speed (meters/second): 0.0000. Beach location: Chicago',
 'output': '2.8690',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context.                   Write a response that appropriately completes the request.\n\n### Instruction:\nWe want to predict 

<a name="Train"></a>
#### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [36]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 0.5e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.02,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/6804 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Zero Shot

In [37]:
# evaluator(serialized_san_test, model)
# evaluator(serialized_chi_test, model)

## Training

In [38]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
11.441 GB of memory reserved.


In [39]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,804 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.597600
2,2.592000
3,2.568600
4,2.569400
5,2.453300
6,2.420700
7,2.374400
8,2.274600
9,2.141400
10,2.046000


In [40]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

482.2756 seconds used for training.
8.04 minutes used for training.
Peak reserved memory = 12.176 GB.
Peak reserved memory for training = 0.735 GB.
Peak reserved memory % of max memory = 82.56 %.
Peak reserved memory for training % of max memory = 4.984 %.


<a name="Inference"></a>
## Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [41]:
r1, g1 = evaluator(serialized_san_test, model)

100%|██████████| 1558/1558 [19:26<00:00,  1.34it/s]

validated generations: 1.0
0.759357023545171 1.0715469243922933 -0.5899076937040655


In [ ]:
r2, g2 = evaluator(serialized_chi_test, model)

  6%|▌         | 229/3710 [02:51<43:33,  1.33it/s]